creating an retrieval-augmented generation (RAG) setup. using embedding models and LLm to do retrieval system

In [97]:
import pdfplumber


def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        document_sections = []
        for page in pdf.pages:
            text = page.extract_text()
            text_segments = text.split("\n\n")  # Split by paragraphs
    
            for text in text_segments:
                if len(text) > 0:
                    document_sections.append(text)
        
    return document_sections

print("Extracting text from PDFs...")

pdf_path1 = "./data/SOFI-2023.pdf"
pdf_path2 = "./data/SOFI-2024.pdf"

document_sections_1 = extract_text_from_pdf(pdf_path1)
document_sections_2 = extract_text_from_pdf(pdf_path2)

document_sections = document_sections_1 + document_sections_2



Extracting text from PDFs...


In [98]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each section
embeddings = model.encode(document_sections)

In [99]:
import faiss
import numpy as np

# Convert embeddings to numpy array
embeddings = np.array(embeddings).astype('float32')

# Initialize Faiss index
dimension = embeddings.shape[1]  # Number of dimensions in the embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [100]:
from transformers import pipeline

In [109]:
def search(query, k=3):
    query_embedding = model.encode([query]).astype('float32')

    # Search for the k most similar embeddings
    _, indices = index.search(query_embedding, k)
    
    # Return the sections
    return [document_sections[idx] for idx in indices[0]]

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

def pipeline1(query, k=3):
    answers = []
    for section in search(query, k):
        result = qa_pipeline(question=query, context=section)
        answers.append(result['answer'])
    
    return ''.join(answers)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [117]:
print(pipeline1("Explain increase prices impact on food security"))

addressing nutrition situation and ability to obtain financingeither on or off the
across the rural–urban continuumMore than one country in the region
surge in international fertilizer prices
